# Imports

In [21]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
import pickle
import json
from pathlib import Path
import joblib
from joblib import dump, load, Parallel, delayed
import os
import itertools
import contextlib
from tqdm import tqdm

# Import Weights Model
import WeightsModel3
from WeightsModel3 import PreProcessing
from WeightsModel3 import RandomForestWeightsModel

# Import (Rolling Horizon) Weighted SAA models
from WeightedSAA6 import WeightedSAA
from WeightedSAA6 import RobustWeightedSAA
from WeightedSAA6 import RollingHorizonOptimization

In [22]:
### Function to concatenate all results
def concatenate_results(path_to_save, name_to_save, SKUs, taus=[None], es=[None]):
    
    results = pd.DataFrame()
    
    # For each product (SKU) k=1,...,M
    for SKU in SKUs:
        
        # For each look-ahead tau=0,...,4
        for tau in taus:
            
            # For each uncertainty set specification  e=1,...,12
            for e in es:
                
                if not e is None:
                    
                    file_name = path_to_save+'/'+name_to_save+'_e'+str(e).replace('.', '')+'_SKU'+str(SKU)+'_tau'+str(tau)+'.csv'
                    
                    # Check if results exist   
                    if os.path.exists(file_name):
                        results = pd.concat([results, pd.read_csv(file_name)])
                    
                else:
                    
                    file_name = path_to_save+'/'+name_to_save+'_SKU'+str(SKU)+'_tau'+str(tau)+'.csv'
            
                    # Check if results exist   
                    if os.path.exists(file_name):
                        results = pd.concat([results, pd.read_csv(file_name)])
    
    return results

In [23]:
def aggregate_results(results, groupby=['CR', 'tau', 'e', 'SKU']):
        
    # Aggregate results over periods t=1,...,T
    results_aggregated = results.groupby(groupby).agg({
        'MIPGap': lambda x: x.iloc[0],
        'NumericFocus': lambda x: x.iloc[0],
        'obj_improvement': lambda x: x.iloc[0],
        'obj_timeout_sec': lambda x: x.iloc[0],
        'obj_timeout_max_sec': lambda x: x.iloc[0],
        'K': lambda x: x.iloc[0],
        'u': lambda x: x.iloc[0],
        'h': lambda x: x.iloc[0],
        'b': lambda x: x.iloc[0],
        'I': np.mean,
        'q': np.mean,
        'I_q': np.mean,
        'y': np.mean,
        'I_q_y': np.mean,
        'c_o': sum,
        'c_s': sum,
        'cost': sum,
        'defaulted': sum,
        'solutions': lambda x: sum(x>0),
        'gap': np.mean,
        'exec_time_sec': np.mean,
        'cpu_time_sec': np.mean
    }).reset_index()

    return results_aggregated

# General paths and parameters

In [24]:
# Set folder names as global variables
os.chdir('/home/fesc/DataDrivenDynamicInventoryControl/')
global PATH_DATA, PATH_WEIGHTSMODEL, PATH_RESULTS

PATH_DATA = '/home/fesc/DataDrivenDynamicInventoryControl/Data' 
PATH_WEIGHTSMODEL = '/home/fesc/DataDrivenDynamicInventoryControl/Data/WeightsModel'
PATH_RESULTS = '/home/fesc/DataDrivenDynamicInventoryControl/Data/Results'

In [25]:
# Time period and SKU ranges
T = 13                  # Planning horizon T
ts = range(1,13+1)      # Periods t=1,...,T of the planning horizon
taus = range(0,4+1)     # Look-aheads tau=0,...,4 to use
es = [1,3,6,9,12]       # Uncertainty set specifications e=1,...,12
SKUs = range(1,460+1)   # Products (SKUs) k=1,...,M

# Train/test split (first timePeriods of testing horizon)
test_start = 114

# Cost param settings
cost_params = [

    {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
    {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
    {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}

]

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
path_to_save_GwSAA = PATH_RESULTS+'/GwSAA'
name_to_save_GwSAA = 'GwSAA'

# Rolling Horizon Global Robust Weighted SAA (GwSAA-R)
path_to_save_GwSAAR = PATH_RESULTS+'/GwSAAR'
name_to_save_GwSAAR = 'GwSAAR'

# Rolling Horizon Local Weighted SAA (wSAA)
path_to_save_wSAA = PATH_RESULTS+'/wSAA'
name_to_save_wSAA = 'wSAA'

# Rolling Horizon Local Robust Weighted SAA (wSAA-R)
path_to_save_wSAAR = PATH_RESULTS+'/wSAAR'
name_to_save_wSAAR = 'wSAAR'

# Rolling Horizon Local SAA (SAA)
path_to_save_SAA = PATH_RESULTS+'/SAA'
name_to_save_SAA = 'SAA'

# Ex-post clairvoyant model
path_to_save_ExPost = PATH_RESULTS+'/ExPost'
name_to_save_ExPost = 'ExPost'

# Evaluation

## Post-processing

### Concatenate and save

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
results = concatenate_results(path_to_save_GwSAA, name_to_save_GwSAA, SKUs, taus)
file_name = path_to_save_GwSAA+'/'+name_to_save_GwSAA+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Rolling Horizon Global Robust Weighted SAA (GwSAA-R)
results = concatenate_results(path_to_save_GwSAAR, name_to_save_GwSAAR, SKUs, taus, es)
file_name = path_to_save_GwSAAR+'/'+name_to_save_GwSAAR+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Rolling Horizon Local Weighted SAA (wSAA)
results = concatenate_results(path_to_save_wSAA, name_to_save_wSAA, SKUs, taus)
file_name = path_to_save_wSAA+'/'+name_to_save_wSAA+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Rolling Horizon Local Robust Weighted SAA (wSAA-R)
results = concatenate_results(path_to_save_wSAAR, name_to_save_wSAAR, SKUs, taus, es)
file_name = path_to_save_wSAAR+'/'+name_to_save_wSAAR+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Rolling Horizon Local SAA (SAA)
results = concatenate_results(path_to_save_SAA, name_to_save_SAA, SKUs, taus)
file_name = path_to_save_SAA+'/'+name_to_save_SAA+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Ex-post clairvoyant model
results = concatenate_results(path_to_save_ExPost, name_to_save_ExPost, SKUs)
file_name = path_to_save_ExPost+'/'+name_to_save_ExPost+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA) - Reshaped, Scaled, New Hyper Params
path_to_save_GwSAA = PATH_RESULTS+'/GwSAA_FINAL'
name_to_save_GwSAA = 'GwSAA_FINAL'

In [ ]:
results = concatenate_results(path_to_save_GwSAA, name_to_save_GwSAA, SKUs, taus)

In [ ]:
results.groupby(['CR', 'tau', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum})

In [50]:
# Rolling Horizon Global Robust Weighted SAA (GwSAAR)
path_to_save_GwSAAR = PATH_RESULTS+'/GwSAAR_FINAL'
name_to_save_GwSAAR = 'GwSAAR_FINAL'

In [51]:
results = concatenate_results(path_to_save_GwSAAR, name_to_save_GwSAAR, SKUs, taus, es)

In [52]:
results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum}).reset_index()

,CR,cost
0,0.50,7196659.0
1,0.75,9023673.0
2,0.90,11007853.0


In [53]:
len(results) # Should be 448500 when done

381849

In [54]:
results.groupby(['e']).agg({'cost': sum})

,cost
e,
1,162781137.0
3,180426737.5
6,265287402.5
9,367630193.0
12,132388378.5


In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
path_to_save_GwSAA = PATH_RESULTS+'/GwSAA_old_params'
name_to_save_GwSAA = 'GwSAA_old_params'

In [ ]:
results = concatenate_results(path_to_save_GwSAA, name_to_save_GwSAA, SKUs, taus)

In [ ]:
results.groupby(['CR', 'tau', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum})

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
path_to_save_GwSAA = PATH_RESULTS+'/GwSAA_r_z_old_params'
name_to_save_GwSAA = 'GwSAA_r_z_old_params'

In [ ]:
results = concatenate_results(path_to_save_GwSAA, name_to_save_GwSAA, SKUs, taus)

In [ ]:
results.groupby(['CR', 'tau', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum})

In [ ]:
# Rolling Horizon Global Robust Weighted SAA (GwSAAR)
path_to_save_GwSAAR = PATH_RESULTS+'/GwSAAR_r_z_old_params'
name_to_save_GwSAAR = 'GwSAAR_r_z_old_params'

In [ ]:
results = concatenate_results(path_to_save_GwSAAR, name_to_save_GwSAAR, SKUs, taus, es)

In [ ]:
results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum}).reset_index()

In [ ]:
results = concatenate_results(path_to_save_GwSAAR, name_to_save_GwSAAR, SKUs, taus, es)

In [ ]:
results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum}).reset_index()

In [ ]:
results = concatenate_results(path_to_save_wSAAR, name_to_save_wSAAR, SKUs, taus, es)

In [ ]:
results.groupby(['CR', 'tau', 'e', 'SKU']).agg({'cost': sum}).reset_index().groupby(['CR', 'SKU']).agg({'cost': min}).reset_index().groupby('CR').agg({'cost': sum}).reset_index()

In [ ]:
samples = joblib.load('/home/fesc/DataDrivenDynamicInventoryControl/Data/WeightsModel/rfwm_global_not_reshaped_old_rf_params_samples_tau0.joblib')

In [ ]:
sum(samples[5]['y_train'])

In [ ]:
sum(samples[5]['X_train'])

In [ ]:
42585913.0 - 44471128.0

In [ ]:
samples[5]['X_train'].shape

In [ ]:
samples[5]['y_train']

### Load

In [ ]:
# Rolling Horizon Global Weighted SAA (GwSAA)
file_name = path_to_save_GwSAA+'/'+name_to_save_GwSAA+'_results.csv'
results_GwSAA = pd.read_csv(file_name)
#results_GwSAA['model'] = 'GwSAA'
#results_GwSAA['e'] = 0

In [ ]:
# Rolling Horizon Global Robust Weighted SAA (GwSAA-R)
file_name = path_to_save_GwSAAR+'/'+name_to_save_GwSAAR+'_results.csv'
results_GwSAAR = pd.read_csv(file_name)
#results_GwSAAR['model'] = 'GwSAA-R'

In [ ]:
# Rolling Horizon Local Weighted SAA (wSAA)
file_name = path_to_save_wSAA+'/'+name_to_save_wSAA+'_results.csv'
results_wSAA = pd.read_csv(file_name)
#results_wSAA['model'] = 'wSAA'
#results_wSAA['e'] = 0

In [ ]:
# Rolling Horizon Local Robust Weighted SAA (wSAA-R)
file_name = path_to_save_wSAAR+'/'+name_to_save_wSAAR+'_results.csv'
results_wSAAR = pd.read_csv(file_name)
#results_wSAAR['model'] = 'wSAA-R'

In [ ]:
# Rolling Horizon Local SAA (SAA)
file_name = path_to_save_SAA+'/'+name_to_save_SAA+'_results.csv'
results_SAA = pd.read_csv(file_name)
#results_SAA['model'] = 'SAA'
#results_SAA['e'] = 0

In [ ]:
# Ex-post clairvoyant model
file_name = path_to_save_ExPost+'/'+name_to_save_ExPost+'_results.csv'
results_ExPost = pd.read_csv(file_name)
#result_ExPost['model'] = 'ExPost'
# result_ExPost['e'] = 0
# results_ExPost = pd.DataFrame()
# for tau in taus:
#     result_ExPost['tau'] = tau
#     results_ExPost = pd.concat([results_ExPost,result_ExPost])

### Aggregate

In [ ]:
# Aggregate results over periods t=1,...,T
results_agg_GwSAA = aggregate_results(results_GwSAA, ['CR', 'tau', 'SKU'])
results_agg_GwSAAR = aggregate_results(results_GwSAAR, ['CR', 'tau', 'e', 'SKU'])
results_agg_wSAA = aggregate_results(results_wSAA, ['CR', 'tau', 'SKU'])
results_agg_wSAAR = aggregate_results(results_wSAAR, ['CR', 'tau', 'e', 'SKU'])
results_agg_SAA = aggregate_results(results_SAA, ['CR', 'tau', 'SKU'])
results_agg_ExPost = aggregate_results(results_ExPost, ['CR', 'SKU'])

In [ ]:
path_to_save_GwSAA_z = PATH_RESULTS+'/GwSAA_NEW_z'
name_to_save_GwSAA_z = 'GwSAA_NEW_z'
tau=1

In [ ]:
results = concatenate_results(path_to_save_GwSAA_z, name_to_save_GwSAA_z, SKUs, taus)

In [ ]:
res_z = results.loc[results.tau==tau].groupby(['CR', 'tau', 'SKU']).agg({'cost': sum, 'y': sum}).reset_index()

In [ ]:
file_name = path_to_save_GwSAA+'/'+name_to_save_GwSAA+'_results.csv'
results_GwSAA = pd.read_csv(file_name)

In [ ]:
res = results_GwSAA.loc[results_GwSAA.tau==tau].groupby(['CR', 'tau', 'SKU']).agg({'cost': sum, 'y': sum}).reset_index()

In [ ]:
test = pd.merge(left=res_z, right=res, on=['CR', 'tau', 'SKU'], suffixes=('_z',''))

In [ ]:
test['diffs'] = test.cost_z / test.cost

In [ ]:
test.describe()

### Select

In [ ]:
#### Best tau per product

In [ ]:
def xxx(results_agg, results_agg_ExPost, groupby = ['CR', 'SKU']):

    """
    
    If groupby == ['CR', 'SKU', 'e']: then looks for best tau per product and per uncertainty set sepcification for given CR
    If groupby == ['CR', 'SKU']: then looks for best tau per product for given CR
    If groupby == ['CR']: looks for best tau across all products for given CR
    If groupby == []: not implemented

    
    """
    
    # Merge aggregated results with ex-post clairvoyant results
    results_best_tau = pd.merge(left=results_agg,
                                right=results_agg_ExPost[['CR', 'SKU', 'cost']],
                                on=['CR', 'SKU'],
                                suffixes=('', '_ExPost'))
    
    # Calculate gap to ex-post clairvoyant results
    results_best_tau['gap'] = (
        (results_best_tau.cost == results_best_tau.cost_ExPost) * 1 
        + (results_best_tau.cost != results_best_tau.cost_ExPost) * (results_best_tau.cost / results_best_tau.cost_ExPost)
    ) - 1
    
    # Calculate median gap per tau
    results_best_tau = results_best_tau.groupby(groupby+['tau']).agg({'gap': np.median}).reset_index()

    # Find tau that minimizes median gap
    results_best_tau = results_best_tau.groupby(groupby).apply(
        lambda df:  pd.Series({ 'best_tau': df.tau.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()
    
    # Merge best tau back to original results for ex-post selection
    results_best_tau = pd.merge(left=results_agg,
                                right=results_best_tau,
                                on=groupby)

    # Select
    results_best_tau = results_best_tau.loc[results_best_tau.tau==results_best_tau.best_tau]

    return results_best_tau

In [ ]:
res = xxx(results_agg_GwSAA, results_agg_ExPost, groupby=['CR'])

In [ ]:
res

In [ ]:
res = xxx(results_agg_wSAA, results_agg_ExPost, groupby=['CR', 'SKU'])
res.groupby(['CR']).agg({'cost': np.median}).reset_index()

In [ ]:
res = xxx(results_agg_GwSAA, results_agg_ExPost, groupby=['CR', 'SKU'])
res.groupby(['CR']).agg({'cost': np.median}).reset_index()

In [ ]:
res = xxx(results_agg_GwSAA, results_agg_ExPost, groupby=['CR', 'SKU'])

In [ ]:
res.groupby(['CR']).agg({'cost': sum}).reset_index()

In [ ]:
12884307.0 / 13177200.0

In [ ]:
res = results_agg_wSAAR.groupby(['CR', 'e', 'tau']).agg({'cost': sum}).reset_index()
res.groupby(['CR']).agg({'cost': min}).reset_index()

In [ ]:
res = results_agg_GwSAA.groupby(['CR', 'tau']).agg({'cost': sum}).reset_index()
res.groupby(['CR']).agg({'cost': min}).reset_index()

In [ ]:
res = results_agg_wSAA.groupby(['CR', 'tau']).agg({'cost': sum}).reset_index()
res.groupby(['CR']).agg({'cost': min}).reset_index()

In [ ]:
res = results_agg_GwSAAR.groupby(['CR', 'e', 'tau']).agg({'cost': sum}).reset_index()
res.groupby(['CR']).agg({'cost': min}).reset_index()

In [ ]:
res = results_agg_SAA.groupby(['CR', 'tau']).agg({'cost': sum}).reset_index()
res.groupby(['CR']).agg({'cost': min}).reset_index()

In [ ]:
14706950.5 / 20963542.5

In [ ]:
13841047.0 / 20963542.5

In [ ]:
0.6602437064250949 / 0.7015489152179313

In [ ]:
0.6602437064250949 - 0.7015489152179313

In [ ]:
1 - 13841047.0 / 14706950.5

In [ ]:
xxx(results_agg_GwSAA, results_agg_ExPost)

In [ ]:
cols = ['CR', 'SKU', 'tau', 'cost']
groupby = ['CR', 'tau']
    
# Merge aggregated results with ex-post clairvoyant results
results_best_tau = pd.merge(left=results_agg_GwSAA[cols],
                            right=results_agg_ExPost[['CR', 'SKU', 'cost']],
                            on=['CR', 'SKU'],
                            suffixes=('', '_ExPost'))

In [ ]:
# Calculate gap to ex-post clairvoyant results
results_best_tau['gap'] = (
    (results_best_tau.cost == results_best_tau.cost_ExPost) * 1 
    + (results_best_tau.cost != results_best_tau.cost_ExPost) * (results_best_tau.cost / results_best_tau.cost_ExPost)
) - 1

# Calculate median gap per tau
results_best_tau = results_best_tau.groupby(groupby).agg({'gap': np.median}).reset_index()

In [ ]:
results_GwSAA_best_tau = pd.merge(left=results_GwSAA.groupby(['CR', 'SKU', 'tau']).agg({'cost': sum}).reset_index(),
                                  right=results_ExPost,
                                  on=['CR', 'SKU', 'tau'],
                                  suffixes=('', '_ExPost'))

results_GwSAA_best_tau['gap'] = (
    (results_GwSAA_best_tau.cost==results_GwSAA_best_tau.cost_ExPost)*1 
    + (results_GwSAA_best_tau.cost!=results_GwSAA_best_tau.cost_ExPost)*(results_GwSAA_best_tau.cost/results_GwSAA_best_tau.cost_ExPost)
)

results_GwSAA_best_tau = results_GwSAA_best_tau.groupby(['CR', 'tau']).agg({'gap': np.median}).reset_index()

results_GwSAA_best_tau = results_GwSAA_best_tau.groupby(['CR']).apply(
    lambda df:  pd.Series({ 'best_tau': df.tau.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

results_GwSAA_best_tau = pd.merge(left=results_GwSAA,
                                  right=results_GwSAA_best_tau,
                                  on=['CR'])

results_GwSAA_best_tau = results_GwSAA_best_tau.loc[results_GwSAA_best_tau.tau==results_GwSAA_best_tau.best_tau]

In [ ]:
cols = results_GwSAA.columns
results = pd.concat([results_GwSAA[cols], results_GwSAAR[cols], results_wSAA[cols], results_wSAAR[cols], results_SAA[cols], results_ExPost[cols]])

In [ ]:
cols = results_GwSAA.columns
results = pd.concat([results_GwSAA[cols], results_GwSAAR_best[cols], results_wSAA[cols], results_wSAAR_best[cols], results_SAA[cols], results_ExPost[cols]])

In [ ]:
# Summarize results
results_summary = results.groupby(['model', 'CR', 'tau', 'e', 'SKU']).agg({
    'MIPGap': lambda x: x.iloc[0],
    'NumericFocus': lambda x: x.iloc[0],
    'obj_improvement': lambda x: x.iloc[0],
    'obj_timeout_sec': lambda x: x.iloc[0],
    'obj_timeout_max_sec': lambda x: x.iloc[0],
    'K': lambda x: x.iloc[0],
    'u': lambda x: x.iloc[0],
    'h': lambda x: x.iloc[0],
    'b': lambda x: x.iloc[0],
    'I': np.mean,
    'q': np.mean,
    'I_q': np.mean,
    'y': np.mean,
    'I_q_y': np.mean,
    'c_o': sum,
    'c_s': sum,
    'cost': sum,
    'defaulted': sum,
    'solutions': lambda x: sum(x>0),
    'gap': np.mean,
    'exec_time_sec': np.mean,
    'cpu_time_sec': np.mean
}).reset_index()

In [ ]:
results_summary

In [ ]:
# Ex-post best results
results_eval = results_summary.groupby(['model', 'CR', 'SKU', 'tau']).agg({
    'cost': min,
}).reset_index().groupby(['model', 'CR', 'SKU']).agg({
    'cost': min,
}).reset_index()
results_eval

In [ ]:
results_eval_baseline = results_eval.loc[results_eval.model == 'SAA',['CR', 'SKU', 'cost']]
results_eval_expost = results_eval.loc[results_eval.model == 'ExPost',['CR', 'SKU', 'cost']]

results_eval = pd.merge(left=results_eval.loc[results_eval.model.isin(['GwSAA', 'GwSAA-R', 'wSAA', 'wSAA-R'])],
                        right=results_eval_baseline,
                        on=['CR', 'SKU'],
                        suffixes=('', '_SAA'))

results_eval = pd.merge(left=results_eval,
                        right=results_eval_expost,
                        on=['CR', 'SKU'],
                        suffixes=('', '_ExPost'))

In [ ]:
results_eval

In [ ]:
results_eval['pq'] = (
    (results_eval.cost == results_eval.cost_SAA)*1 
    + (results_eval.cost != results_eval.cost_SAA)*results_eval.cost/results_eval.cost_SAA
)

In [ ]:
results_eval.describe()

In [ ]:
results_eval.groupby(['CR', 'model']).agg({'pq': np.median}).reset_index()

In [ ]:
totals = results_eval.groupby(['CR', 'model']).agg({'cost': sum, 'cost_SAA': sum}).reset_index()

In [ ]:
totals['pq'] = totals.cost / totals.cost_SAA

In [ ]:
totals

In [ ]:
results_GwSAAR_best = pd.merge(left=results_GwSAAR.groupby(['CR', 'SKU', 'tau', 'e']).agg({'cost': sum}).reset_index(),
                               right=results_ExPost,
                               on=['CR', 'SKU', 'tau'],
                               suffixes=('', '_ExPost'))

results_GwSAAR_best['gap'] = (
    (results_GwSAAR_best.cost==results_GwSAAR_best.cost_ExPost)*1 
    + (results_GwSAAR_best.cost!=results_GwSAAR_best.cost_ExPost)*(results_GwSAAR_best.cost/results_GwSAAR_best.cost_ExPost)
)

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR', 'tau', 'e']).agg({'gap': np.median}).reset_index()

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR', 'tau']).apply(
    lambda df:  pd.Series({ 'best_e': df.e.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

In [ ]:
results_GwSAAR_best

In [ ]:
results_GwSAAR_best = pd.merge(left=results_GwSAAR,
                              right=results_GwSAAR_best,
                              on=['CR', 'tau'])

results_GwSAAR_best = results_GwSAAR_best.loc[results_GwSAAR_best.e==results_GwSAAR_best.best_e]

In [ ]:
results_wSAAR_best = pd.merge(left=results_wSAAR.groupby(['CR', 'SKU', 'tau', 'e']).agg({'cost': sum}).reset_index(),
                              right=results_ExPost,
                              on=['CR', 'SKU', 'tau'],
                              suffixes=('', '_ExPost'))

results_wSAAR_best['gap'] = (
    (results_wSAAR_best.cost==results_wSAAR_best.cost_ExPost)*1 
    + (results_wSAAR_best.cost!=results_wSAAR_best.cost_ExPost)*(results_wSAAR_best.cost/results_wSAAR_best.cost_ExPost)
)

results_wSAAR_best = results_wSAAR_best.groupby(['CR', 'tau', 'e']).agg({'gap': np.median}).reset_index()

results_wSAAR_best = results_wSAAR_best.groupby(['CR', 'tau']).apply(
    lambda df:  pd.Series({ 'best_e': df.e.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

In [ ]:
results_wSAAR_best

In [ ]:
results_wSAAR_best = pd.merge(left=results_wSAAR,
                              right=results_wSAAR_best,
                              on=['CR', 'tau'])

results_wSAAR_best = results_wSAAR_best.loc[results_wSAAR_best.e==results_wSAAR_best.best_e]

In [ ]:
#### Choose ex-post tau

In [ ]:
## Best per product

In [ ]:
## Best aross all

In [ ]:
results_GwSAA_best_tau = pd.merge(left=results_GwSAA.groupby(['CR', 'SKU', 'tau']).agg({'cost': sum}).reset_index(),
                                  right=results_ExPost,
                                  on=['CR', 'SKU', 'tau'],
                                  suffixes=('', '_ExPost'))

results_GwSAA_best_tau['gap'] = (
    (results_GwSAA_best_tau.cost==results_GwSAA_best_tau.cost_ExPost)*1 
    + (results_GwSAA_best_tau.cost!=results_GwSAA_best_tau.cost_ExPost)*(results_GwSAA_best_tau.cost/results_GwSAA_best_tau.cost_ExPost)
)

results_GwSAA_best_tau = results_GwSAA_best_tau.groupby(['CR', 'tau']).agg({'gap': np.median}).reset_index()

results_GwSAA_best_tau = results_GwSAA_best_tau.groupby(['CR']).apply(
    lambda df:  pd.Series({ 'best_tau': df.tau.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

results_GwSAA_best_tau = pd.merge(left=results_GwSAA,
                                  right=results_GwSAA_best_tau,
                                  on=['CR'])

results_GwSAA_best_tau = results_GwSAA_best_tau.loc[results_GwSAA_best_tau.tau==results_GwSAA_best_tau.best_tau]

In [ ]:
results_wSAA_best_tau = pd.merge(left=results_wSAA.groupby(['CR', 'SKU', 'tau']).agg({'cost': sum}).reset_index(),
                                  right=results_ExPost,
                                  on=['CR', 'SKU', 'tau'],
                                  suffixes=('', '_ExPost'))

results_wSAA_best_tau['gap'] = (
    (results_wSAA_best_tau.cost==results_wSAA_best_tau.cost_ExPost)*1 
    + (results_wSAA_best_tau.cost!=results_wSAA_best_tau.cost_ExPost)*(results_wSAA_best_tau.cost/results_wSAA_best_tau.cost_ExPost)
)

results_wSAA_best_tau = results_wSAA_best_tau.groupby(['CR', 'tau']).agg({'gap': np.median}).reset_index()

results_wSAA_best_tau = results_wSAA_best_tau.groupby(['CR']).apply(
    lambda df:  pd.Series({ 'best_tau': df.tau.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

results_wSAA_best_tau = pd.merge(left=results_wSAA,
                                  right=results_wSAA_best_tau,
                                  on=['CR'])

results_wSAA_best_tau = results_wSAA_best_tau.loc[results_wSAA_best_tau.tau==results_wSAA_best_tau.best_tau]

In [ ]:
results_GwSAAR_best = pd.merge(left=results_GwSAAR.groupby(['CR', 'SKU', 'tau', 'e']).agg({'cost': sum}).reset_index(),
                               right=results_ExPost,
                               on=['CR', 'SKU', 'tau'],
                               suffixes=('', '_ExPost'))

results_GwSAAR_best['gap'] = (
    (results_GwSAAR_best.cost==results_GwSAAR_best.cost_ExPost)*1 
    + (results_GwSAAR_best.cost!=results_GwSAAR_best.cost_ExPost)*(results_GwSAAR_best.cost/results_GwSAAR_best.cost_ExPost)
)

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR', 'tau', 'e']).agg({'gap': np.median}).reset_index()

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR']).apply(
    lambda df:  pd.Series({'best_tau': df.tau.iloc[np.argmin(df.gap)], 
                           'best_e': df.e.iloc[np.argmin(df.gap)], 
                           'best_gap': np.min(df.gap)})).reset_index()


In [ ]:

results_GwSAAR_best = pd.merge(left=results_GwSAAR,
                              right=results_GwSAAR_best,
                              on=['CR', 'tau'])

results_GwSAAR_best = results_GwSAAR_best.loc[results_GwSAAR_best.e==results_GwSAAR_best.best_e]

In [ ]:
results_GwSAAR_best

In [ ]:
fixed_params = results_summary.loc[(results_summary.tau==4) & (results_summary.e <= 1)]

In [ ]:
fixed_params.groupby(['CR', 'model']).agg({'cost': sum}).reset_index()

In [ ]:
#### Choose ex-post e

In [ ]:
## Best per product

In [ ]:
## Best aross all

In [ ]:
results_GwSAAR_best = pd.merge(left=results_GwSAAR.groupby(['CR', 'SKU', 'tau', 'e']).agg({'cost': sum}).reset_index(),
                               right=results_ExPost,
                               on=['CR', 'SKU', 'tau'],
                               suffixes=('', '_ExPost'))

results_GwSAAR_best['gap'] = (
    (results_GwSAAR_best.cost==results_GwSAAR_best.cost_ExPost)*1 
    + (results_GwSAAR_best.cost!=results_GwSAAR_best.cost_ExPost)*(results_GwSAAR_best.cost/results_GwSAAR_best.cost_ExPost)
)

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR', 'tau', 'e']).agg({'gap': np.median}).reset_index()

results_GwSAAR_best = results_GwSAAR_best.groupby(['CR', 'tau']).apply(
    lambda df:  pd.Series({ 'best_e': df.e.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

results_GwSAAR_best = pd.merge(left=results_GwSAAR,
                              right=results_GwSAAR_best,
                              on=['CR', 'tau'])

results_GwSAAR_best = results_GwSAAR_best.loc[results_GwSAAR_best.e==results_GwSAAR_best.best_e]

In [ ]:
results_wSAAR_best = pd.merge(left=results_wSAAR.groupby(['CR', 'SKU', 'tau', 'e']).agg({'cost': sum}).reset_index(),
                              right=results_ExPost,
                              on=['CR', 'SKU', 'tau'],
                              suffixes=('', '_ExPost'))

results_wSAAR_best['gap'] = (
    (results_wSAAR_best.cost==results_wSAAR_best.cost_ExPost)*1 
    + (results_wSAAR_best.cost!=results_wSAAR_best.cost_ExPost)*(results_wSAAR_best.cost/results_wSAAR_best.cost_ExPost)
)

results_wSAAR_best = results_wSAAR_best.groupby(['CR', 'tau', 'e']).agg({'gap': np.median}).reset_index()

results_wSAAR_best = results_wSAAR_best.groupby(['CR', 'tau']).apply(
    lambda df:  pd.Series({ 'best_e': df.e.iloc[np.argmin(df.gap)], 'best_gap': np.min(df.gap)})).reset_index()

In [ ]:
results_wSAAR_best = pd.merge(left=results_wSAAR,
                              right=results_wSAAR_best,
                              on=['CR', 'tau'])

results_wSAAR_best = results_wSAAR_best.loc[results_wSAAR_best.e==results_wSAAR_best.best_e]

In [ ]:
results_summary.groupby(['CR', 'model', 'tau']).agg({'cost': sum}).reset_index()

In [ ]:
results_summary.groupby(['CR', 'model', 'tau']).agg({'cost': sum}).reset_index().groupby(['CR', 'model']).agg({'cost': min}).reset_index()

In [ ]:
results_summary.groupby(['CR', 'model', 'tau']).agg({'cost': sum}).reset_index().groupby(['CR', 'model']).agg({'cost': np.argmin}).reset_index()

In [ ]:
# Aggregate results
results = aggregateResults(taus, SKUs, experiment_params['path_to_save'], experiment_params['name_to_save'])
results

In [ ]:
# Save aggregate results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results.csv'
results.to_csv(file_name, sep=',', index=False)

In [ ]:
# Summarize results
results_summary = results.groupby(['SKU', 'CR', 'tau']).agg({
    'MIPGap': lambda x: x.iloc[0],
    'NumericFocus': lambda x: x.iloc[0],
    'obj_improvement': lambda x: x.iloc[0],
    'obj_timeout_sec': lambda x: x.iloc[0],
    'obj_timeout_max_sec': lambda x: x.iloc[0],
    'K': lambda x: x.iloc[0],
    'u': lambda x: x.iloc[0],
    'h': lambda x: x.iloc[0],
    'b': lambda x: x.iloc[0],
    'I': np.mean,
    'q': np.mean,
    'I_q': np.mean,
    'y': np.mean,
    'I_q_y': np.mean,
    'c_o': sum,
    'c_s': sum,
    'cost': sum,
    'defaulted': sum,
    'solutions': lambda x: sum(x>0),
    'gap': np.mean,
    'exec_time_sec': np.mean,
    'cpu_time_sec': np.mean
}).reset_index()
results_summary

In [ ]:
# Save summarized results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results_summary.csv'
results_summary.to_csv(file_name, sep=',', index=False)

In [ ]:
# Evaluate results
results_evaluation = results_summary.groupby(['SKU', 'CR']).agg({
    'cost': min
}).reset_index()
results_evaluation

In [ ]:
# Save evaluated results
file_name = experiment_params['path_to_save']+'/'+experiment_params['name_to_save']+'_results_evaluation.csv'
results_evaluation.to_csv(file_name, sep=',', index=False)